In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("ETL Pipeline - Bronze to Silver") \
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

In [4]:
bronze_path = "s3a://my-bucket-ry-01/raw-data/ipca/kafka/" 

In [6]:
df_bronze = spark.read.json(bronze_path)

Py4JJavaError: An error occurred while calling o35.json.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(Da

In [7]:
df_bronze.show()

+-----------+-------------+---------------+-----------+-------------+------------+----+----------+-------------+---------+
|CompraManha|    Data_Base|Data_Vencimento|PUBaseManha|PUCompraManha|PUVendaManha|Tipo|VendaManha|    dt_update|partition|
+-----------+-------------+---------------+-----------+-------------+------------+----+----------+-------------+---------+
|       6.35|1719187200000|  1786752000000|    3753.72|      3764.16|     3753.72|IPCA|      6.47|1743228344967|        0|
|      10.27|1719187200000|  1723680000000|    4230.97|      4233.79|     4230.97|IPCA|     10.39|1743228344968|        0|
|      -1.29|1556755200000|  1557878400000|    3219.03|      3219.66|     3219.54|IPCA|     -1.17|1743228349406|        0|
|       4.43|1556582400000|  2378419200000|    1013.05|      1044.31|     1013.64|IPCA|      4.55|1743228349407|        0|
|      -1.49|1556236800000|  1557878400000|    3215.88|      3217.83|     3217.66|IPCA|     -1.37|1743228349417|        0|
|       4.16|155

In [8]:
from pyspark.sql.functions import col, from_unixtime

In [9]:
df_silver = df_bronze.dropDuplicates()

In [10]:
df_silver = df_silver.withColumn("Data_Vencimento", from_unixtime(col("Data_Vencimento") / 1000, "yyyy-MM-dd")) \
.withColumn("Data_Base", from_unixtime(col("Data_Base") / 1000, "yyyy-MM-dd")) \
.withColumn("dt_update", from_unixtime(col("dt_update") / 1000, "yyyy-MM-dd HH:mm:ss"))

In [11]:
df_silver = df_silver.fillna({"PUCompraManha": 0, "PUVendaManha": 0, "PUBaseManha": 0}) 

In [12]:
df_silver.show()

+-----------+----------+---------------+-----------+-------------+------------+----+----------+-------------------+---------+
|CompraManha| Data_Base|Data_Vencimento|PUBaseManha|PUCompraManha|PUVendaManha|Tipo|VendaManha|          dt_update|partition|
+-----------+----------+---------------+-----------+-------------+------------+----+----------+-------------------+---------+
|      -1.58|2019-05-10|     2019-05-15|     3222.4|      3224.08|     3224.04|IPCA|     -1.46|2025-03-29 06:05:49|        0|
|      -0.69|2015-03-20|     2015-05-15|    2584.96|      2588.42|     2588.34|IPCA|     -0.67|2025-03-29 06:05:52|        0|
|       6.16|2024-09-09|     2035-05-15|    2261.44|      2289.26|     2261.44|IPCA|      6.28|2025-03-29 06:05:44|        0|
|      -0.54|2019-04-24|     2019-05-15|    3212.55|      3213.27|     3213.07|IPCA|     -0.42|2025-03-29 06:05:49|        0|
|      10.18|2024-06-18|     2024-08-15|    4222.05|      4224.91|     4222.05|IPCA|      10.3|2025-03-29 06:05:44|   

In [13]:
silver_path = "s3a://my-bucket-carlao-01/processed-data/ipca/silver/"

In [14]:
df_silver.write.mode("overwrite").parquet(silver_path)

In [15]:
from pyspark.sql.functions import avg, count

In [17]:
df_gold = df_silver.groupBy("Tipo").agg(avg("PUCompraManha").alias("Media_PUCompraManha"), avg("PUVendaManha").alias("Media_PUVendaManha"), count("*").alias("Total_Registros"))

In [18]:
df_gold.show()

+----+-------------------+------------------+---------------+
|Tipo|Media_PUCompraManha|Media_PUVendaManha|Total_Registros|
+----+-------------------+------------------+---------------+
|IPCA|  1771.582629854369|1757.3804830097047|          16480|
+----+-------------------+------------------+---------------+



In [21]:
gold_path = "s3a://my-bucket-carlao-01/analytics/ipca/gold/"

In [22]:
df_gold.write.mode("overwrite").parquet(gold_path)

In [23]:
bronze_columns = df_bronze.columns

In [24]:
bronze_columns

['CompraManha',
 'Data_Base',
 'Data_Vencimento',
 'PUBaseManha',
 'PUCompraManha',
 'PUVendaManha',
 'Tipo',
 'VendaManha',
 'dt_update',
 'partition']

In [26]:
df_bronze.groupBy(bronze_columns).count().filter("count > 1").show()

+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+-----+
|CompraManha|Data_Base|Data_Vencimento|PUBaseManha|PUCompraManha|PUVendaManha|Tipo|VendaManha|dt_update|partition|count|
+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+-----+
+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+-----+



In [31]:
df_bronze.filter(
    col("PUCompraManha").isNull() |
    col("PUVendaManha").isNull() |
    col("PUBaseManha").isNull()
).show()

+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+
|CompraManha|Data_Base|Data_Vencimento|PUBaseManha|PUCompraManha|PUVendaManha|Tipo|VendaManha|dt_update|partition|
+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+
+-----------+---------+---------------+-----------+-------------+------------+----+----------+---------+---------+

